In [1]:
from pathlib import Path
import yaml
import obspy
import pandas as pd
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt

PROJECT_ROOT = Path.cwd().parent

with open(PROJECT_ROOT / "config.yaml", "r") as f:
    cfg = yaml.safe_load(f)

DATA_ROOT = Path(cfg["paths"]["data_root"])
LABELED_DATA = Path(cfg["paths"]["labeled_data"])
UNLABELED_DATA = Path(cfg["paths"]["unlabeled_data"])

FIGURES_DIR = PROJECT_ROOT / cfg["paths"]["figures_output"]
TABLES_DIR = PROJECT_ROOT / cfg["paths"]["tables_output"]
NOTEBOOKS_DIR = PROJECT_ROOT / "notebooks"

FIGURES_DIR.mkdir(exist_ok=True)
TABLES_DIR.mkdir(exist_ok=True)

print("PROJECT_ROOT :", PROJECT_ROOT)
print("FIGURES_DIR  :", FIGURES_DIR.resolve())
print("TABLES_DIR   :", TABLES_DIR.resolve())


PROJECT_ROOT : c:\Users\Disc\Desktop\pj
FIGURES_DIR  : C:\Users\Disc\Desktop\pj\figures
TABLES_DIR   : C:\Users\Disc\Desktop\pj\tables


In [2]:
n_files = sum(1 for p in UNLABELED_DATA.rglob("*") if p.is_file())
print(f"Archivos en UNLABELED_DATA: {n_files}")


Archivos en UNLABELED_DATA: 13761


In [3]:
from pathlib import Path
import pandas as pd

files = [p.name for p in UNLABELED_DATA.rglob("*") if p.is_file()]

records = []
for f in files:
    parts = f.split(".")
    if len(parts) == 6:
        net, sta, loc, chan, year, doy = parts
        records.append({
            "network": net,
            "station": sta,
            "location": loc,
            "channel": chan,
            "year": int(year),
            "doy": int(doy),
            "filename": f
        })

df = pd.DataFrame(records)
print(df.head())


  network station location channel  year  doy                 filename
0      CL    LC03       01     HHZ  2021  141  CL.LC03.01.HHZ.2021.141
1      CL    LC03       01     HHZ  2021  151  CL.LC03.01.HHZ.2021.151
2      CL    LC03       01     HHN  2021  141  CL.LC03.01.HHN.2021.141
3      CL    LC03       01     HHN  2021  151  CL.LC03.01.HHN.2021.151
4      CL    LC03       01     HHE  2021  141  CL.LC03.01.HHE.2021.141


In [4]:
stations = sorted(df["station"].unique())
print("Estaciones:")
for s in stations:
    print(" -", s)


Estaciones:
 - LC01
 - LC02
 - LC03
 - LC04
 - LC05
 - LC06
 - LC07
 - LC08
 - LC09
 - LC10
 - LC11


In [5]:
events_per_station = df.groupby("station").size().sort_values(ascending=False)
print(events_per_station)


station
LC03    3321
LC01    3180
LC02    2661
LC04    1914
LC06     450
LC08     450
LC09     447
LC10     447
LC07     429
LC11     279
LC05     183
dtype: int64


In [6]:
events_per_channel = df.groupby("channel").size()
print(events_per_channel)


channel
HHE    4587
HHN    4587
HHZ    4587
dtype: int64


In [7]:
station_channel = df.groupby(["station", "channel"]).size().unstack(fill_value=0)
print(station_channel)


channel   HHE   HHN   HHZ
station                  
LC01     1060  1060  1060
LC02      887   887   887
LC03     1107  1107  1107
LC04      638   638   638
LC05       61    61    61
LC06      150   150   150
LC07      143   143   143
LC08      150   150   150
LC09      149   149   149
LC10      149   149   149
LC11       93    93    93


In [8]:
events_per_year = df.groupby("year").size().sort_index()
print(events_per_year)


year
2018    4200
2021    1956
2022    3447
2023    3363
2024     795
dtype: int64


In [9]:
events_station_year = (
    df.groupby(["year", "station"])
      .size()
      .unstack(fill_value=0)
      .sort_index()
)

print(events_station_year)


station  LC01  LC02  LC03  LC04  LC05  LC06  LC07  LC08  LC09  LC10  LC11
year                                                                     
2018      333     6   483   693   183   450   429   450   447   447   279
2021      654   654   648     0     0     0     0     0     0     0     0
2022      948   945   945   609     0     0     0     0     0     0     0
2023     1095   810  1095   363     0     0     0     0     0     0     0
2024      150   246   150   249     0     0     0     0     0     0     0


In [10]:
events_full = (
    df[df["channel"].isin(["HHE", "HHN", "HHZ"])]
    .groupby(["year", "station", "channel"])
    .size()
    .unstack(fill_value=0)
    .sort_index()
)

print(events_full)


channel       HHE  HHN  HHZ
year station               
2018 LC01     111  111  111
     LC02       2    2    2
     LC03     161  161  161
     LC04     231  231  231
     LC05      61   61   61
     LC06     150  150  150
     LC07     143  143  143
     LC08     150  150  150
     LC09     149  149  149
     LC10     149  149  149
     LC11      93   93   93
2021 LC01     218  218  218
     LC02     218  218  218
     LC03     216  216  216
2022 LC01     316  316  316
     LC02     315  315  315
     LC03     315  315  315
     LC04     203  203  203
2023 LC01     365  365  365
     LC02     270  270  270
     LC03     365  365  365
     LC04     121  121  121
2024 LC01      50   50   50
     LC02      82   82   82
     LC03      50   50   50
     LC04      83   83   83


In [11]:
subset_df = df[
    (df["station"] == "LC01") &
    (df["year"].isin([2022, 2023])) &
    (df["channel"].isin(["HHE", "HHN", "HHZ"]))
].copy()

print(subset_df.groupby(["year", "channel"]).size())


year  channel
2022  HHE        316
      HHN        316
      HHZ        316
2023  HHE        365
      HHN        365
      HHZ        365
dtype: int64


In [12]:
from obspy import read
import pandas as pd
import numpy as np
from tqdm import tqdm

records = []

# Filtrar explícitamente lo que quieres
hzz_df = subset_df[
    (subset_df["channel"] == "HHZ")
].copy()

for _, row in tqdm(
    hzz_df.iterrows(),
    total=len(hzz_df),
    desc="Leyendo HHZ LC01 2022–2023"
):
    file_path = UNLABELED_DATA / row["filename"]

    try:
        st = read(file_path)
        tr = st[0]

        # vector de tiempos
        times = tr.times("datetime")
        data = tr.data.astype(float)

        # guardar pares fecha–dato
        for t, v in zip(times, data):
            records.append({
                "datetime": t,
                "value": v
            })

    except Exception as e:
        print(f"Error leyendo {row['filename']}: {e}")


Leyendo HHZ LC01 2022–2023:   0%|          | 2/681 [00:00<04:29,  2.52it/s]

Error leyendo CL.LC01.01.HHZ.2022.001: Invalid `type`: datetime
Error leyendo CL.LC01.01.HHZ.2022.002: Invalid `type`: datetime


Leyendo HHZ LC01 2022–2023:   1%|          | 4/681 [00:01<02:57,  3.82it/s]

Error leyendo CL.LC01.01.HHZ.2022.003: Invalid `type`: datetime
Error leyendo CL.LC01.01.HHZ.2022.004: Invalid `type`: datetime


Leyendo HHZ LC01 2022–2023:   1%|          | 5/681 [00:01<02:40,  4.22it/s]

...

Leyendo HHZ LC01 2022–2023:   6%|▋         | 43/681 [00:09<02:12,  4.81it/s]

Error leyendo CL.LC01.01.HHZ.2022.042: Invalid `type`: datetime
Error leyendo CL.LC01.01.HHZ.2022.043: Invalid `type`: datetime


Leyendo HHZ LC01 2022–2023:   6%|▋         | 44/681 [00:09<02:19,  4.56it/s]

Error leyendo CL.LC01.01.HHZ.2022.044: Invalid `type`: datetime


KeyboardInterrupt: 

In [14]:
tr.times

<bound method Trace.times of <obspy.core.trace.Trace object at 0x000001722E967410>>

In [29]:
from obspy import read
import pandas as pd

# 👉 cambia esto por el nombre real del archivo
filename = "CL.LC01.01.HHZ.2022.123"

file_path = UNLABELED_DATA / filename

# Leer archivo
st = read(file_path)
st.decimate(factor=10, no_filter=True)  # 200 Hz → 20 Hz
st.write('outfile.ascii', format='TSPAIR')

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8640000 entries, 0 to 8639999
Data columns (total 2 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   date    object
 1   value   int64 
dtypes: int64(1), object(1)
memory usage: 131.8+ MB


In [24]:
# Extraer tiempo y señal
times = tr.times("utcdatetime")
values = tr.data.astype(float)

# Crear DataFrame
df_out = pd.DataFrame({
    "datetime": times,
    "value": values
})

# Guardar CSV
out_csv = TABLES_DIR / "test_LC01_HHZ_single_day.csv"
df_out.to_csv(out_csv, index=False)

print("CSV creado:", out_csv)
print(df_out.head())


CSV creado: c:\Users\Disc\Desktop\pj\tables\test_LC01_HHZ_single_day.csv
                      datetime    value
0  2022-05-03T00:00:00.000000Z  70512.0
1  2022-05-03T00:00:00.010000Z  70510.0
2  2022-05-03T00:00:00.020000Z  70509.0
3  2022-05-03T00:00:00.030000Z  70505.0
4  2022-05-03T00:00:00.040000Z  70502.0
